In [1]:
### basic package for data science project
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import helpers

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
sns.set()

In [2]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [3]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [95]:
import numpy as np
from scipy.stats import randint
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [150]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, auc, precision_recall_curve, roc_curve, average_precision_score

### Helpers

In [103]:
def reporting(ensem_preds, targets):
    best_th = 0
    best_score = 0

    for th in np.arange(0.0, 0.6, 0.01):
        pred = (ensem_preds > th).astype(int)
        score = f1_score(targets, pred)
        if score > best_score:
            best_th = th
            best_score = score

    print(f"\nAUC score: {roc_auc_score(targets, ensem_preds):12.4f}")
    print(f"Best threshold {best_th:12.4f}")

    preds = (ensem_preds > best_th).astype(int)
    # print(classification_report(targets, preds, digits=3))

    cm1 = confusion_matrix(targets, preds)
    print('\nConfusion Matrix : \n', cm1)
    total1=sum(sum(cm1))

    print('\n=============')
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print (f'Accuracy    : {accuracy1:12.4f}')

    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print(f'Sensitivity : {sensitivity1:12.4f}')

    specificity1 = cm1[1,1]/(cm1[0,1]+cm1[1,1])
    print(f'Specificity : {specificity1:12.4f}')

1. Remove columns with missing value more than 75% 
2. Remove col 'x39' and 'x99' since those two columns don't bring additional information 
3. Impute NAN for missing values in numerical methods 
4. We can use Catboost or Imputing categorical features is to replace missing values with the most common class. 


Alternative (if time permitted):
    - Check each features to see if there exists outliers.

In [137]:
train_data = pd.read_csv('data/exercise_40_train.csv')
test_data = pd.read_csv('data/exercise_40_test.csv')

In [227]:
most_missing_cols = list(set(train_data.columns[train_data.isnull().mean() > 0.75]))
remove_cat_col = ['x39', 'x99']
remove_cols = most_missing_cols + remove_cat_col

In [228]:
def create_dummy_df(df, cat_cols, dummy_na):
    for col in  cat_cols:
        try:
            # for each cat add dummy var, drop original column
            df = pd.concat([df.drop(col, axis=1),\
                            pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True, dummy_na=dummy_na)], axis=1)
        except:
            continue
    return df

def clean_catergorical_ft(df):
    
    cat_df = df.select_dtypes(include=['object'])
    
    #### clean col x7 and x19
    cat_df['x7'] = cat_df['x7'].str.replace('%','')
    cat_df["x19"] = cat_df['x19'].str.replace('$','')
    cat_df = cat_df.astype({'x7': 'float', 'x19': 'float'})

    #### clean x3 col
    cat_df['x3'] = cat_df.apply(lambda x: helpers.clean_day_x3_col(x['x3']), axis = 1)

    ### convert x7 and x19 back to numerical fts 
    df['x7'] =  cat_df['x7']
    df['x19'] = cat_df["x19"]

    #### categorical cols 
    cat_df.drop(['x7','x19'], axis = 1, inplace = True)
    cat_cols = cat_df.columns
    
    return cat_cols, cat_df, df
    

In [152]:
def ft_engineering_test(df, remove_cols, impute_num_path = 'impute_mean.pickle', use_catboost = True):
    df = df.loc[:, (~df.columns.isin(remove_cols))]
    print(df.shape)
    #### clean categorical features
    cat_cols, cat_df, df = clean_catergorical_ft(df)
    
    #### clean numerical features 
    numerical_df = df.loc[:, (~df.columns.isin(cat_cols))]

    ### imputing missing values 
    with open(impute_num_path, 'rb') as f:
        imp_mean = pickle.load(f)
    num_df_trasformed = pd.DataFrame(imp_mean.transform(numerical_df.loc[:, ~(numerical_df.columns.isin(["y"]))]))
    
    
    ### if not using catboost model, then imputing categorical features 
    ### by replacing missing values with the most common class
    if use_catboost == False:
        
        ### replace missing value with most common value
        for col in cat_cols:
            max_freq = cat_df[col].value_counts().index[0]
            cat_df[col][pd.isna(cat_df[col])] = max_freq
    
        cat_df_transformed = create_dummy_df(cat_df, cat_cols, dummy_na= True)

    else:
        cat_df_transformed = cat_df.fillna('NaN')
        num_df_trasformed = numerical_df.fillna('NaN')
       
    cleaned_data = pd.concat([num_df_trasformed, cat_df_transformed.reset_index(drop = True)], axis=1)

    cleaned_data["y"] = df['y'].reset_index(drop = True)

    return cleaned_data


In [284]:
def ft_engineering_train(df, remove_cols, use_catboost = True):
    df = df.loc[:, (~df.columns.isin(remove_cols))]
 
    #### clean categorical features
    cat_cols, cat_df, df = clean_catergorical_ft(df)
    
    #### clean numerical features 
    numerical_df = df.loc[:, (~df.columns.isin(cat_cols))]
            
    ### if not using catboost model, then imputing categorical features 
    ### by replacing missing values with the most common class
    if use_catboost == False:
        ### imputing missing values 
#         imp_mean = IterativeImputer(random_state=0)
#         num_df_trasformed = pd.DataFrame(imp_mean.fit_transform(numerical_df.loc[:, ~(numerical_df.columns.isin(["y"]))]))
        num_df_trasformed = numerical_df.fillna(0)
        ## save labelEncoder for test set
        with open('impute_mean.pickle', 'wb') as f:
            pickle.dump(imp_mean, f, pickle.HIGHEST_PROTOCOL)
        ### replace missing value with most common value
        for col in cat_cols:
            max_freq = cat_df[col].value_counts().index[0] 
            cat_df[col][pd.isna(cat_df[col])] = max_freq
            
        one_hot_encoder = OneHotEncoder()
        cat_df_transformed = pd.DataFrame(one_hot_encoder.fit_transform(cat_df).toarray())
        with open('one_hot_encoder.pickle', 'wb') as f:
            pickle.dump(one_hot_encoder, f, pickle.HIGHEST_PROTOCOL)
      
    else:
        cat_df_transformed = cat_df.fillna('NaN')
        num_df_trasformed = numerical_df.fillna('NaN')
    
    cleaned_data = pd.concat([num_df_trasformed, cat_df_transformed.reset_index(drop = True)], axis=1)

    cleaned_data["y"] = df['y'].reset_index(drop = True)

    return cleaned_data


### Using catboost 

In [280]:
data_cb = ft_engineering_train(train_data, remove_cols, use_catboost = True)

In [215]:
# target
y = data_cb['y']

# features

X = data_cb.loc[:, ~data_cb.columns.isin(['y'])]

#Instantiate CatBoostClassifier
cbc = CatBoostClassifier(class_weights={0:1, 1:6})

# Creating the hyperparameter grid
param_dist = { "learning_rate": np.linspace(0,0.2,5)
               "max_depth": randint(3, 10)}
               
#Instantiate RandomSearchCV object
rscv = RandomizedSearchCV(cbc , param_dist, scoring='accuracy', cv =5)


#Fit the model
categorical_features_indices = np.where(X.dtypes != np.float)[0]
rscv.fit(X,y,cat_features=categorical_features_indices, plot = True)

# Print the tuned parameters and score
print(rscv.best_params_)
print(rscv.best_score_)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6522973	total: 91.6ms	remaining: 1m 31s
1:	learn: 0.6159894	total: 115ms	remaining: 57.5s
2:	learn: 0.5986636	total: 143ms	remaining: 47.4s
3:	learn: 0.5881728	total: 170ms	remaining: 42.2s
4:	learn: 0.5781868	total: 195ms	remaining: 38.8s
5:	learn: 0.5706438	total: 225ms	remaining: 37.2s
6:	learn: 0.5638038	total: 261ms	remaining: 37s
7:	learn: 0.5598696	total: 325ms	remaining: 40.3s
8:	learn: 0.5575162	total: 360ms	remaining: 39.7s
9:	learn: 0.5549310	total: 385ms	remaining: 38.1s
10:	learn: 0.5517220	total: 412ms	remaining: 37.1s
11:	learn: 0.5500642	total: 439ms	remaining: 36.1s
12:	learn: 0.5487395	total: 464ms	remaining: 35.2s
13:	learn: 0.5476690	total: 490ms	remaining: 34.5s
14:	learn: 0.5458192	total: 559ms	remaining: 36.7s
15:	learn: 0.5446358	total: 583ms	remaining: 35.9s
16:	learn: 0.5431811	total: 609ms	remaining: 35.2s
17:	learn: 0.5421855	total: 636ms	remaining: 34.7s
18:	learn: 0.5408222	total: 663ms	remaining: 34.2s
19:	learn: 0.5391958	total: 688ms	remaini

161:	learn: 0.4353682	total: 6.07s	remaining: 31.4s
162:	learn: 0.4349888	total: 6.1s	remaining: 31.3s
163:	learn: 0.4344165	total: 6.13s	remaining: 31.3s
164:	learn: 0.4337660	total: 6.16s	remaining: 31.2s
165:	learn: 0.4333614	total: 6.19s	remaining: 31.1s
166:	learn: 0.4327454	total: 6.22s	remaining: 31s
167:	learn: 0.4321034	total: 6.26s	remaining: 31s
168:	learn: 0.4313943	total: 6.3s	remaining: 31s
169:	learn: 0.4308121	total: 6.33s	remaining: 30.9s
170:	learn: 0.4301060	total: 6.36s	remaining: 30.8s
171:	learn: 0.4295718	total: 6.39s	remaining: 30.7s
172:	learn: 0.4290824	total: 6.42s	remaining: 30.7s
173:	learn: 0.4285421	total: 6.44s	remaining: 30.6s
174:	learn: 0.4279900	total: 6.47s	remaining: 30.5s
175:	learn: 0.4273571	total: 6.56s	remaining: 30.7s
176:	learn: 0.4268140	total: 6.58s	remaining: 30.6s
177:	learn: 0.4262168	total: 6.61s	remaining: 30.5s
178:	learn: 0.4256530	total: 6.64s	remaining: 30.4s
179:	learn: 0.4251190	total: 6.66s	remaining: 30.4s
180:	learn: 0.424447

322:	learn: 0.3562882	total: 12.1s	remaining: 25.4s
323:	learn: 0.3557453	total: 12.2s	remaining: 25.4s
324:	learn: 0.3552796	total: 12.2s	remaining: 25.3s
325:	learn: 0.3550099	total: 12.2s	remaining: 25.2s
326:	learn: 0.3545385	total: 12.2s	remaining: 25.2s
327:	learn: 0.3541345	total: 12.3s	remaining: 25.1s
328:	learn: 0.3538976	total: 12.3s	remaining: 25.1s
329:	learn: 0.3535430	total: 12.4s	remaining: 25.2s
330:	learn: 0.3531218	total: 12.4s	remaining: 25.1s
331:	learn: 0.3526075	total: 12.5s	remaining: 25.1s
332:	learn: 0.3521552	total: 12.5s	remaining: 25s
333:	learn: 0.3517846	total: 12.5s	remaining: 24.9s
334:	learn: 0.3514300	total: 12.5s	remaining: 24.9s
335:	learn: 0.3509217	total: 12.6s	remaining: 24.8s
336:	learn: 0.3505384	total: 12.6s	remaining: 24.8s
337:	learn: 0.3500604	total: 12.6s	remaining: 24.8s
338:	learn: 0.3495603	total: 12.7s	remaining: 24.7s
339:	learn: 0.3490604	total: 12.7s	remaining: 24.6s
340:	learn: 0.3485334	total: 12.7s	remaining: 24.6s
341:	learn: 0.

483:	learn: 0.2970854	total: 18.1s	remaining: 19.3s
484:	learn: 0.2967087	total: 18.2s	remaining: 19.3s
485:	learn: 0.2964074	total: 18.2s	remaining: 19.2s
486:	learn: 0.2960469	total: 18.2s	remaining: 19.2s
487:	learn: 0.2957279	total: 18.2s	remaining: 19.1s
488:	learn: 0.2953949	total: 18.3s	remaining: 19.1s
489:	learn: 0.2950257	total: 18.3s	remaining: 19.1s
490:	learn: 0.2947266	total: 18.5s	remaining: 19.1s
491:	learn: 0.2943904	total: 18.5s	remaining: 19.1s
492:	learn: 0.2940254	total: 18.5s	remaining: 19.1s
493:	learn: 0.2936733	total: 18.6s	remaining: 19s
494:	learn: 0.2931233	total: 18.6s	remaining: 19s
495:	learn: 0.2927319	total: 18.6s	remaining: 18.9s
496:	learn: 0.2924978	total: 18.7s	remaining: 18.9s
497:	learn: 0.2921564	total: 18.8s	remaining: 18.9s
498:	learn: 0.2917997	total: 18.8s	remaining: 18.9s
499:	learn: 0.2914833	total: 18.8s	remaining: 18.8s
500:	learn: 0.2912138	total: 18.8s	remaining: 18.8s
501:	learn: 0.2908994	total: 18.9s	remaining: 18.7s
502:	learn: 0.29

...

323:	learn: 0.3812665	total: 14.5s	remaining: 30.3s
324:	learn: 0.3808085	total: 14.5s	remaining: 30.2s
325:	learn: 0.3804437	total: 14.6s	remaining: 30.1s
326:	learn: 0.3800977	total: 14.6s	remaining: 30s
327:	learn: 0.3797284	total: 14.6s	remaining: 30s
328:	learn: 0.3794826	total: 14.7s	remaining: 29.9s
329:	learn: 0.3791393	total: 14.8s	remaining: 30s
330:	learn: 0.3788127	total: 14.8s	remaining: 29.9s
331:	learn: 0.3784815	total: 14.8s	remaining: 29.9s
332:	learn: 0.3781575	total: 14.9s	remaining: 29.8s
333:	learn: 0.3780206	total: 14.9s	remaining: 29.8s
334:	learn: 0.3776194	total: 15s	remaining: 29.7s
335:	learn: 0.3772500	total: 15s	remaining: 29.7s
336:	learn: 0.3768829	total: 15.1s	remaining: 29.7s
337:	learn: 0.3765181	total: 15.1s	remaining: 29.7s
338:	learn: 0.3763693	total: 15.2s	remaining: 29.6s
339:	learn: 0.3759993	total: 15.2s	remaining: 29.5s
340:	learn: 0.3756271	total: 15.3s	remaining: 29.5s
341:	learn: 0.3752162	total: 15.3s	remaining: 29.4s
342:	learn: 0.3747599	

482:	learn: 0.3285545	total: 21.8s	remaining: 23.4s
483:	learn: 0.3282280	total: 21.9s	remaining: 23.4s
484:	learn: 0.3278723	total: 21.9s	remaining: 23.3s
485:	learn: 0.3275863	total: 22s	remaining: 23.2s
486:	learn: 0.3272821	total: 22s	remaining: 23.2s
487:	learn: 0.3268722	total: 22s	remaining: 23.1s
488:	learn: 0.3265078	total: 22.1s	remaining: 23.1s
489:	learn: 0.3261705	total: 22.1s	remaining: 23s
490:	learn: 0.3259118	total: 22.2s	remaining: 23s
491:	learn: 0.3257700	total: 22.3s	remaining: 23s
492:	learn: 0.3255085	total: 22.3s	remaining: 22.9s
493:	learn: 0.3252399	total: 22.4s	remaining: 22.9s
494:	learn: 0.3252345	total: 22.4s	remaining: 22.8s
495:	learn: 0.3248836	total: 22.4s	remaining: 22.8s
496:	learn: 0.3246018	total: 22.5s	remaining: 22.7s
497:	learn: 0.3242329	total: 22.6s	remaining: 22.7s
498:	learn: 0.3239725	total: 22.6s	remaining: 22.7s
499:	learn: 0.3236671	total: 22.6s	remaining: 22.6s
500:	learn: 0.3233442	total: 22.7s	remaining: 22.6s
501:	learn: 0.3230503	to

643:	learn: 0.2837670	total: 29.3s	remaining: 16.2s
644:	learn: 0.2834722	total: 29.4s	remaining: 16.2s
645:	learn: 0.2832229	total: 29.4s	remaining: 16.1s
646:	learn: 0.2829190	total: 29.4s	remaining: 16.1s
647:	learn: 0.2825883	total: 29.5s	remaining: 16s
648:	learn: 0.2822760	total: 29.5s	remaining: 16s
649:	learn: 0.2819796	total: 29.5s	remaining: 15.9s
650:	learn: 0.2817614	total: 29.6s	remaining: 15.9s
651:	learn: 0.2814347	total: 29.7s	remaining: 15.8s
652:	learn: 0.2811309	total: 29.7s	remaining: 15.8s
653:	learn: 0.2809447	total: 29.8s	remaining: 15.7s
654:	learn: 0.2805137	total: 29.8s	remaining: 15.7s
655:	learn: 0.2801659	total: 29.8s	remaining: 15.6s
656:	learn: 0.2798465	total: 29.9s	remaining: 15.6s
657:	learn: 0.2796732	total: 29.9s	remaining: 15.5s
658:	learn: 0.2794528	total: 30s	remaining: 15.5s
659:	learn: 0.2791032	total: 30s	remaining: 15.5s
660:	learn: 0.2788287	total: 30.1s	remaining: 15.4s
661:	learn: 0.2785356	total: 30.1s	remaining: 15.4s
662:	learn: 0.278162

805:	learn: 0.2453523	total: 37s	remaining: 8.9s
806:	learn: 0.2451865	total: 37s	remaining: 8.85s
807:	learn: 0.2449976	total: 37.1s	remaining: 8.8s
808:	learn: 0.2447908	total: 37.1s	remaining: 8.76s
809:	learn: 0.2445877	total: 37.1s	remaining: 8.71s
810:	learn: 0.2443591	total: 37.1s	remaining: 8.66s
811:	learn: 0.2441249	total: 37.2s	remaining: 8.61s
812:	learn: 0.2438628	total: 37.3s	remaining: 8.58s
813:	learn: 0.2436774	total: 37.3s	remaining: 8.53s
814:	learn: 0.2434042	total: 37.4s	remaining: 8.48s
815:	learn: 0.2432016	total: 37.4s	remaining: 8.43s
816:	learn: 0.2431204	total: 37.4s	remaining: 8.38s
817:	learn: 0.2430630	total: 37.5s	remaining: 8.34s
818:	learn: 0.2428527	total: 37.5s	remaining: 8.29s
819:	learn: 0.2426169	total: 37.6s	remaining: 8.25s
820:	learn: 0.2424381	total: 37.6s	remaining: 8.2s
821:	learn: 0.2422411	total: 37.7s	remaining: 8.16s
822:	learn: 0.2420351	total: 37.7s	remaining: 8.11s
823:	learn: 0.2417901	total: 37.7s	remaining: 8.06s
824:	learn: 0.24156

967:	learn: 0.2142721	total: 44.3s	remaining: 1.46s
968:	learn: 0.2140605	total: 44.3s	remaining: 1.42s
969:	learn: 0.2139746	total: 44.3s	remaining: 1.37s
970:	learn: 0.2138004	total: 44.4s	remaining: 1.32s
971:	learn: 0.2136301	total: 44.4s	remaining: 1.28s
972:	learn: 0.2134345	total: 44.4s	remaining: 1.23s
973:	learn: 0.2132115	total: 44.5s	remaining: 1.19s
974:	learn: 0.2130358	total: 44.5s	remaining: 1.14s
975:	learn: 0.2129546	total: 44.6s	remaining: 1.1s
976:	learn: 0.2127456	total: 44.6s	remaining: 1.05s
977:	learn: 0.2125982	total: 44.7s	remaining: 1s
978:	learn: 0.2124517	total: 44.7s	remaining: 959ms
979:	learn: 0.2122641	total: 44.7s	remaining: 913ms
980:	learn: 0.2120800	total: 44.8s	remaining: 869ms
981:	learn: 0.2118617	total: 44.9s	remaining: 823ms
982:	learn: 0.2117073	total: 44.9s	remaining: 777ms
983:	learn: 0.2115062	total: 44.9s	remaining: 731ms
984:	learn: 0.2112106	total: 45s	remaining: 685ms
985:	learn: 0.2110324	total: 45s	remaining: 639ms
986:	learn: 0.210838

In [281]:
X_train, X_test, y_train, y_test = train_test_split(
                                    data_cb.loc[:,~data_cb.columns.isin(['y'])],
                                    data_cb['y'],
                                    test_size=0.1,
                                    random_state=0)

# categorical_features_indices = np.where(X_train.dtypes != np.float)[0]
categorical_features_indices = [87, 88, 89, 90, 91, 92, 93, 94]
cbc = CatBoostClassifier(learning_rate= 0.05, max_depth= 6, n_estimators= 1000,\
                         cat_features=categorical_features_indices)
cbc.fit(X_train,y_train)

0:	learn: 0.6564345	total: 98.3ms	remaining: 1m 38s
1:	learn: 0.6267303	total: 147ms	remaining: 1m 13s
2:	learn: 0.5996303	total: 203ms	remaining: 1m 7s
3:	learn: 0.5746875	total: 292ms	remaining: 1m 12s
4:	learn: 0.5534777	total: 355ms	remaining: 1m 10s
5:	learn: 0.5329531	total: 410ms	remaining: 1m 7s
6:	learn: 0.5152606	total: 512ms	remaining: 1m 12s
7:	learn: 0.5014111	total: 575ms	remaining: 1m 11s
8:	learn: 0.4889783	total: 627ms	remaining: 1m 9s
9:	learn: 0.4773299	total: 737ms	remaining: 1m 12s
10:	learn: 0.4665493	total: 784ms	remaining: 1m 10s
11:	learn: 0.4564231	total: 879ms	remaining: 1m 12s
12:	learn: 0.4451254	total: 987ms	remaining: 1m 14s
13:	learn: 0.4375465	total: 1.07s	remaining: 1m 15s
14:	learn: 0.4289094	total: 1.14s	remaining: 1m 14s
15:	learn: 0.4219858	total: 1.25s	remaining: 1m 16s
16:	learn: 0.4163414	total: 1.31s	remaining: 1m 15s
17:	learn: 0.4108443	total: 1.37s	remaining: 1m 14s
18:	learn: 0.4069193	total: 1.47s	remaining: 1m 15s
19:	learn: 0.4018422	tot

161:	learn: 0.3235884	total: 11.7s	remaining: 1m
162:	learn: 0.3234867	total: 11.7s	remaining: 1m
163:	learn: 0.3233595	total: 11.8s	remaining: 1m
164:	learn: 0.3232143	total: 11.9s	remaining: 1m
165:	learn: 0.3230778	total: 11.9s	remaining: 1m
166:	learn: 0.3228953	total: 12s	remaining: 1m
167:	learn: 0.3228079	total: 12.1s	remaining: 1m
168:	learn: 0.3226390	total: 12.2s	remaining: 59.9s
169:	learn: 0.3224987	total: 12.3s	remaining: 60s
170:	learn: 0.3223284	total: 12.3s	remaining: 59.8s
171:	learn: 0.3222135	total: 12.4s	remaining: 59.7s
172:	learn: 0.3221097	total: 12.5s	remaining: 59.7s
173:	learn: 0.3220056	total: 12.5s	remaining: 59.5s
174:	learn: 0.3218969	total: 12.6s	remaining: 59.3s
175:	learn: 0.3218375	total: 12.7s	remaining: 59.4s
176:	learn: 0.3216803	total: 12.7s	remaining: 59.3s
177:	learn: 0.3215426	total: 12.8s	remaining: 59.1s
178:	learn: 0.3214111	total: 12.9s	remaining: 59.1s
179:	learn: 0.3212271	total: 12.9s	remaining: 58.9s
180:	learn: 0.3211493	total: 13s	rema

324:	learn: 0.3014298	total: 22.9s	remaining: 47.7s
325:	learn: 0.3013746	total: 23s	remaining: 47.6s
326:	learn: 0.3012050	total: 23.1s	remaining: 47.5s
327:	learn: 0.3010924	total: 23.1s	remaining: 47.4s
328:	learn: 0.3009089	total: 23.2s	remaining: 47.3s
329:	learn: 0.3007486	total: 23.3s	remaining: 47.2s
330:	learn: 0.3005754	total: 23.3s	remaining: 47.1s
331:	learn: 0.3004284	total: 23.4s	remaining: 47.1s
332:	learn: 0.3002756	total: 23.5s	remaining: 47s
333:	learn: 0.3001202	total: 23.5s	remaining: 46.9s
334:	learn: 0.3000087	total: 23.6s	remaining: 46.9s
335:	learn: 0.2998759	total: 23.7s	remaining: 46.8s
336:	learn: 0.2997020	total: 23.7s	remaining: 46.7s
337:	learn: 0.2996034	total: 23.8s	remaining: 46.6s
338:	learn: 0.2995035	total: 23.9s	remaining: 46.5s
339:	learn: 0.2993406	total: 23.9s	remaining: 46.4s
340:	learn: 0.2992305	total: 24s	remaining: 46.4s
341:	learn: 0.2991304	total: 24.1s	remaining: 46.3s
342:	learn: 0.2990071	total: 24.1s	remaining: 46.2s
343:	learn: 0.2988

485:	learn: 0.2816138	total: 34s	remaining: 35.9s
486:	learn: 0.2814588	total: 34s	remaining: 35.9s
487:	learn: 0.2813451	total: 34.1s	remaining: 35.8s
488:	learn: 0.2812146	total: 34.2s	remaining: 35.7s
489:	learn: 0.2811736	total: 34.2s	remaining: 35.6s
490:	learn: 0.2810549	total: 34.3s	remaining: 35.6s
491:	learn: 0.2809991	total: 34.4s	remaining: 35.5s
492:	learn: 0.2808306	total: 34.4s	remaining: 35.4s
493:	learn: 0.2807047	total: 34.5s	remaining: 35.4s
494:	learn: 0.2805927	total: 34.6s	remaining: 35.3s
495:	learn: 0.2804802	total: 34.6s	remaining: 35.2s
496:	learn: 0.2803364	total: 34.7s	remaining: 35.2s
497:	learn: 0.2801938	total: 34.8s	remaining: 35.1s
498:	learn: 0.2800404	total: 34.8s	remaining: 35s
499:	learn: 0.2799166	total: 34.9s	remaining: 34.9s
500:	learn: 0.2797916	total: 35s	remaining: 34.8s
501:	learn: 0.2796902	total: 35s	remaining: 34.8s
502:	learn: 0.2795895	total: 35.1s	remaining: 34.7s
503:	learn: 0.2794809	total: 35.2s	remaining: 34.6s
504:	learn: 0.2793315	

644:	learn: 0.2636951	total: 45.2s	remaining: 24.9s
645:	learn: 0.2636098	total: 45.3s	remaining: 24.8s
646:	learn: 0.2634645	total: 45.4s	remaining: 24.8s
647:	learn: 0.2634183	total: 45.4s	remaining: 24.7s
648:	learn: 0.2633024	total: 45.5s	remaining: 24.6s
649:	learn: 0.2632084	total: 45.6s	remaining: 24.5s
650:	learn: 0.2630534	total: 45.6s	remaining: 24.5s
651:	learn: 0.2629162	total: 45.7s	remaining: 24.4s
652:	learn: 0.2627722	total: 45.8s	remaining: 24.3s
653:	learn: 0.2626808	total: 45.8s	remaining: 24.2s
654:	learn: 0.2625979	total: 45.9s	remaining: 24.2s
655:	learn: 0.2624779	total: 46s	remaining: 24.1s
656:	learn: 0.2622973	total: 46.1s	remaining: 24.1s
657:	learn: 0.2622201	total: 46.1s	remaining: 24s
658:	learn: 0.2621293	total: 46.2s	remaining: 23.9s
659:	learn: 0.2620041	total: 46.3s	remaining: 23.8s
660:	learn: 0.2618857	total: 46.3s	remaining: 23.8s
661:	learn: 0.2617774	total: 46.4s	remaining: 23.7s
662:	learn: 0.2616595	total: 46.5s	remaining: 23.6s
663:	learn: 0.26

804:	learn: 0.2471470	total: 56.2s	remaining: 13.6s
805:	learn: 0.2470457	total: 56.3s	remaining: 13.5s
806:	learn: 0.2469238	total: 56.3s	remaining: 13.5s
807:	learn: 0.2468399	total: 56.4s	remaining: 13.4s
808:	learn: 0.2466889	total: 56.5s	remaining: 13.3s
809:	learn: 0.2466091	total: 56.6s	remaining: 13.3s
810:	learn: 0.2465416	total: 56.6s	remaining: 13.2s
811:	learn: 0.2464210	total: 56.7s	remaining: 13.1s
812:	learn: 0.2463334	total: 56.7s	remaining: 13.1s
813:	learn: 0.2462277	total: 56.8s	remaining: 13s
814:	learn: 0.2461260	total: 57s	remaining: 12.9s
815:	learn: 0.2459783	total: 57s	remaining: 12.9s
816:	learn: 0.2458600	total: 57.1s	remaining: 12.8s
817:	learn: 0.2457645	total: 57.1s	remaining: 12.7s
818:	learn: 0.2456556	total: 57.2s	remaining: 12.6s
819:	learn: 0.2456058	total: 57.3s	remaining: 12.6s
820:	learn: 0.2454884	total: 57.4s	remaining: 12.5s
821:	learn: 0.2453491	total: 57.4s	remaining: 12.4s
822:	learn: 0.2452408	total: 57.5s	remaining: 12.4s
823:	learn: 0.2450

965:	learn: 0.2317639	total: 1m 7s	remaining: 2.38s
966:	learn: 0.2316609	total: 1m 7s	remaining: 2.31s
967:	learn: 0.2315394	total: 1m 7s	remaining: 2.24s
968:	learn: 0.2315017	total: 1m 7s	remaining: 2.17s
969:	learn: 0.2314045	total: 1m 7s	remaining: 2.1s
970:	learn: 0.2313651	total: 1m 8s	remaining: 2.03s
971:	learn: 0.2313020	total: 1m 8s	remaining: 1.96s
972:	learn: 0.2312098	total: 1m 8s	remaining: 1.89s
973:	learn: 0.2311120	total: 1m 8s	remaining: 1.82s
974:	learn: 0.2310164	total: 1m 8s	remaining: 1.75s
975:	learn: 0.2309344	total: 1m 8s	remaining: 1.68s
976:	learn: 0.2308453	total: 1m 8s	remaining: 1.61s
977:	learn: 0.2307571	total: 1m 8s	remaining: 1.54s
978:	learn: 0.2306998	total: 1m 8s	remaining: 1.47s
979:	learn: 0.2306443	total: 1m 8s	remaining: 1.4s
980:	learn: 0.2305386	total: 1m 8s	remaining: 1.33s
981:	learn: 0.2304759	total: 1m 8s	remaining: 1.26s
982:	learn: 0.2303758	total: 1m 8s	remaining: 1.19s
983:	learn: 0.2302773	total: 1m 8s	remaining: 1.12s
984:	learn: 0.

In [282]:
reporting(cbc.predict_proba(X_test)[:, 1], y_test)


AUC score:       0.8196
Best threshold       0.2200

Confusion Matrix : 
 [[2899  497]
 [ 257  347]]

Accuracy    :       0.8115
Sensitivity :       0.9186
Specificity :       0.4111


### Logistic Regression 

In [ ]:
data = ft_engineering_train(train_data, remove_cols, use_catboost = False)

In [206]:
scaler = StandardScaler()
scaler.fit(X_train)

clf = LogisticRegression(random_state=0, C = 0.2)

In [207]:
scores = cross_val_score(clf, X_train, y_train, cv=5)
scores

array([0.85602941, 0.85617647, 0.85617647, 0.85632353, 0.85617647])

In [208]:
clf.fit(X_train, y_train)

LogisticRegression(C=0.2, random_state=0)

In [209]:
#Generate predicted probabilites
clf_probs = clf.predict_proba(X_test)
print('AUC: ', roc_auc_score(y_test, clf_probs[:,1]))
print('Accuracy: ', clf.score(X_test, y_test))

AUC:  0.5856340636775528
Accuracy:  0.8478333333333333


In [217]:
reporting(clf.predict_proba(X_test)[:,1], y_test)


AUC score:       0.5856
Best threshold       0.1300

Confusion Matrix : 
 [[2354 2733]
 [ 311  602]]

Accuracy    :       0.4927
Sensitivity :       0.4627
Specificity :       0.6594


In [177]:
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import RFECV

score = 'roc_auc'
#Setup recursive feature reduction w/ cross validation
clf2 = RFECV(LogisticRegression(),
      scoring = score,
      n_jobs = -1,
      cv = 3,
      step = 5)
clf2.fit(X_train, y_train)


RFECV(cv=3, estimator=LogisticRegression(), n_jobs=-1, scoring='roc_auc',
      step=5)

In [178]:
#Generate predicted probabilites
clf2_probs = clf2.predict_proba(X_test)
print('AUC: ', roc_auc_score(y_test, clf2_probs[:,1]))
print('Accuracy: ', clf2.score(X_test, y_test))

AUC:  0.6947079200875199
Accuracy:  0.8478333333333333


### Removing features using lasso

In [185]:
sel_ = SelectFromModel(LogisticRegression(penalty='l1', solver='saga', C = 0.1))
sel_.fit(scaler.transform(X_train), y_train)

SelectFromModel(estimator=LogisticRegression(C=0.1, penalty='l1',
                                             solver='saga'))

In [186]:
selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

total features: 116
selected features: 100
features with coefficients shrank to zero: 15


In [187]:
removed_feats = X_train.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
removed_feats

Index(['x22', 'x26', 'x48', 'x59', 'x69', 'x72', 'x79', 'x81', 'x88', 'x90',
       'x24_male', 'x60_January', 'x65_esurance', 'x77_chevrolet', 'x77_ford'],
      dtype='object')

In [188]:
X_train_selected = sel_.transform(X_train)
X_test_selected = sel_.transform(X_test)

In [189]:
clf_selected = LogisticRegression(random_state=0, C = 0.2)

In [190]:
scores = cross_val_score(clf_selected, X_train, y_train, cv=5)
scores

array([0.85602941, 0.85617647, 0.85617647, 0.85632353, 0.85617647])

In [193]:
clf_selected.fit(X_train_selected, y_train)

LogisticRegression(C=0.2, random_state=0)

In [194]:
#Generate predicted probabilites
clf_probs = clf_selected.predict_proba(X_test_selected)
print('AUC: ', roc_auc_score(y_test, clf_probs[:,1]))
print('Accuracy: ', clf_selected.score(X_test_selected, y_test))

AUC:  0.6067447228734801
Accuracy:  0.8476666666666667


In [196]:
reporting(clf_selected.predict_proba(X_test_selected)[:,1], y_test)


AUC score:       0.6067
Best threshold       0.1500

Confusion Matrix : 
 [[3222 1865]
 [ 434  479]]

Accuracy    :       0.6168
Sensitivity :       0.6334
Specificity :       0.5246


## Ensemble 

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import ExtraTreesClassifier

class Model(BaseEstimator, ClassifierMixin):  
    """mixed of lgb, xgb, et"""

    def __init__(self, seed=0, nest_lgb=1.0, nest_xgb=1.0, nest_et=1.0, cbt=0.75, ss=0.75, alpha=0.5, pos_scale=1.):
        """
        Top 3 tree models
        """
        self.classes_ = [0,1]
        self.models = [
                         lgb.LGBMClassifier(class_weight='balanced', 
                                            num_leaves=31, 
                                            max_depth=-1, 
                                            min_child_samples=2, 
                                            learning_rate=0.02,
                                            n_estimators=int(100*nest_lgb), 
                                            colsample_bytree=cbt, 
                                            subsample=ss, 
                                            n_jobs=-1, 
                                            random_state=0+seed
                                           ),
                         lgb.LGBMClassifier(class_weight=None, 
                                            num_leaves=31, 
                                            max_depth=-1, 
                                            min_child_samples=2, 
                                            learning_rate=0.01,
                                            n_estimators=int(200*nest_lgb), 
                                            colsample_bytree=cbt, 
                                            subsample=ss, 
                                            n_jobs=-1, 
                                            random_state=1+seed
                                           ),
                         xgb.XGBClassifier(max_depth=12,
                                           scale_pos_weight=1.,
                                           learning_rate=0.01, 
                                           n_estimators=int(100 * nest_xgb),
                                           subsample=ss, # 0.75
                                           colsample_bytree=cbt, # 0.75
                                           nthread=-1,
                                           seed=0+seed,
                                           eval_metric='logloss',
                                           use_label_encoder=False
                                          ),
                         xgb.XGBClassifier(max_depth=6,
                                           scale_pos_weight=pos_scale,
                                           learning_rate=0.01,
                                           n_estimators=int(200 * nest_xgb),
                                           subsample=ss, # 0.75
                                           colsample_bytree=cbt, # 0.75
                                           nthread=-1,
                                           seed=1+seed,
                                           eval_metric='logloss',
                                           use_label_encoder=False
                                          ),
                         ExtraTreesClassifier(class_weight='balanced', 
                                              bootstrap=False, 
                                              criterion='entropy', 
                                              max_features=cbt, 
                                              min_samples_leaf=4, 
                                              min_samples_split=3, 
                                              n_estimators= int(100 * nest_et), 
                                              random_state=0+seed, 
                                              n_jobs=-1),
                         
                      ]
        self.weights = [(1-alpha)*1, (1-alpha)*1, (1-alpha)*1, (1-alpha)*1, (1-alpha)*1]


    def fit(self, X, y=None):
        for t, clf in enumerate(self.models):
            clf.fit(X, y)
        return self

    def predict(self, X):
        suma = 0.0
        for t, clf in enumerate(self.models):
            a = clf.predict_proba(X)[:, 1]
            suma += (self.weights[t] * a)
        return (suma / sum(self.weights))
            
    def predict_proba(self, X):      
        return (self.predict(X))